In [34]:
import torch
import torch.nn as nn
import numpy as np

In [35]:
torch.manual_seed(0)

In [36]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [38]:
char_set = list(set(sentence))
char_dic = {c:i for i,c in enumerate(char_set)}
char_dic

{' ': 5,
 "'": 13,
 ',': 10,
 '.': 15,
 'a': 18,
 'b': 2,
 'c': 8,
 'd': 20,
 'e': 9,
 'f': 14,
 'g': 6,
 'h': 24,
 'i': 1,
 'k': 22,
 'l': 11,
 'm': 16,
 'n': 0,
 'o': 12,
 'p': 4,
 'r': 21,
 's': 19,
 't': 7,
 'u': 23,
 'w': 17,
 'y': 3}

In [39]:
dic_size = len(char_dic)
hidden_size = len(char_dic)
sequence_length = 10
learning_rate = 0.1

In [41]:
x_data=[]
y_data=[]

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1:i+sequence_length+1]
  print(i, x_str, '->', y_str)
  
  x_data.append([char_dic[c] for c in x_str])
  y_data.append([char_dic[c] for c in y_str])

x_one_hot = [np.eye(dic_size)[x] for x in x_data]

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [44]:
x_one_hot

[array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [45]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
y = torch.LongTensor(y_data)

In [46]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim,num_layers=layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, hidden_dim, bias = True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

net = Net(dic_size, hidden_size,2)

In [58]:
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), learning_rate)

for i in range(100):
  outputs = net(X)
  cost = criterion(outputs.view(-1, dic_size), y.view(-1))

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  results = outputs.argmax(dim=2)
  predict_str = ""
  for j, result in enumerate(results):
        # print(i, j, ''.join([char_set[t] for t in result]), loss.item())
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
            # print('sdfsdf',result)
        else:
            predict_str += char_set[result[-1]]

  print(predict_str)
  # print(outputs)
  # print(result)
  # print(np.squeeze(result))

sdfsdf tensor([11,  5,  3, 12, 23,  5, 17, 18,  0,  7])
l you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
sdfsdf tensor([16,  0,  3, 12, 23,  5,  5, 18,  0,  7])
mnyou  ant wa cuild tnship, don't wrum up p,opl  thg them tollollect wood dnd tot't wsiign them tysss and dook, but wathir toa h them thf on  for to mp d oss ummensity tf themp an
sdfsdf tensor([11,  0,  3, 12, 21,  8, 23, 18,  0,  7])
lnyorcuant toecuild a ship, drust urum up oeopleetoeethe  toelollect iood and don't uckign the  tosks and dork, bri rather tyachelhe  to long for the  ndless immensity of the eeas
sdfsdf tensor([14,  5,  3, 12, 21,  5, 17, 12,  0,  7])
f yor wont toecuitd asship, don't daum u  peopleeeorethe  to bollect doodland o n't dssign thsm tosks a d dork, ari rat er toach them to long for thecendless immensity of the eeac
sdfsdf tensor([14,  5,  3, 12, 21,  5, 17, 12,  0,  7])


In [54]:
outputs = net(X)

In [55]:
outputs.view(-1, dic_size)

tensor([[  1.0573,   1.3500,   3.3890,  ...,  -5.9212,   4.5019, -10.9438],
        [  6.3107,  -6.4902,  -9.2699,  ...,  -8.5439,  -6.3324,  -1.9829],
        [ -2.6958,  -2.9765,   4.5233,  ...,  -5.1749,   0.9859,  -5.7635],
        ...,
        [  1.5902,   0.3935, -13.4547,  ...,  -4.3755,  -2.0681,   7.7991],
        [  4.7841,   1.8978,  -9.4533,  ...,   0.0196,   1.4921,   4.5838],
        [  3.4511,   4.0921,  -1.1184,  ...,  -0.0664,   0.1360,  -3.6323]],
       grad_fn=<ViewBackward>)

In [56]:
outputs

tensor([[[  1.0573,   1.3500,   3.3890,  ...,  -5.9212,   4.5019, -10.9438],
         [  6.3107,  -6.4902,  -9.2699,  ...,  -8.5439,  -6.3324,  -1.9829],
         [ -2.6958,  -2.9765,   4.5233,  ...,  -5.1749,   0.9859,  -5.7635],
         ...,
         [  1.5330,   2.8249,  -0.1343,  ...,   3.3453,   4.8126,   4.5390],
         [ 15.9863,   6.0485, -13.3582,  ...,  -7.1107,   5.8184,   1.9458],
         [ -5.7274,  -3.3017,   0.8896,  ...,   3.7053,  -6.3288,  -3.8161]],

        [[  0.2796,   5.8654,  -8.6461,  ...,  -3.2979,   9.8823,   6.4522],
         [  5.7871,   4.3290,  -6.3385,  ...,  -4.8304,   1.9961,  -5.5562],
         [ -1.8200,  -6.1877,  -1.3284,  ...,  -2.1811,  -2.6848,   5.1266],
         ...,
         [ 16.7210,   6.6811, -14.4678,  ...,  -8.1186,   6.0392,   1.9963],
         [ -5.6578,  -3.3245,   1.1783,  ...,   3.4746,  -6.5191,  -4.6744],
         [ -4.9585,  -6.1153, -10.8723,  ...,  -3.0679,  -9.5723,   4.9915]],

        [[ -1.9856,   4.0130,   4.7956,  ...